<a href="https://colab.research.google.com/github/clear21/Answer-Mashine/blob/master/RNN_jeopardy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import urllib.request
import json
import pandas as pd
import numpy as np

from keras.layers import Dense , Activation , SimpleRNN , LSTM , Embedding
from keras.models import Sequential

from datetime import datetime

import re

In [0]:
#■関数

#◆jsonファイルで保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_json(obj , fname):
    json_fname_with_extension = fname + '.json'
    fw = open(json_fname_with_extension , 'w')
    json.dump(obj , fw , indent = 4)
    fw.close()
    files.download(json_fname_with_extension)

#◆jsonファイルを取得
#引数：対象ファイルのパス
def import_json(fpass):
    fr = open(fpass , 'r')
    f = json.load(fr)
    fr.close()
    return f

In [2]:
#QAデータセットの取得（JEOPARDY_CSV.csv）
files.upload()

# qa_data_url = r'https://drive.google.com/file/d/0BwT5wj_P7BKXUl9tOUJWYzVvUjA/view'
# qa_file_name = 'JEOPARDY_CSV.csv'
# urllib.request.urlretrieve(qa_data_url , qa_file_name)

Saving JEOPARDY_CSV.csv to JEOPARDY_CSV.csv


In [3]:
#QAデータをデータフレーム化（Question と Answer を使用する）
qa_csv = r'JEOPARDY_CSV.csv'
qa_df = pd.read_csv(qa_csv)[[' Question' , ' Answer']]

#画像URLありの質問　＆　Answerが１wordでない　を除外
qa_df = qa_df[~qa_df[' Question'].str.contains('<a href=')]
print('画像URLありの質問' , '除外後' , len(qa_df))

qa_df[' Answer'] = qa_df[' Answer'].str.strip()
qa_df = qa_df[~qa_df[' Answer'].str.contains(' ' , na = True)]
print('Answerが１wordでない' , '除外後' , len(qa_df))
qa_df = qa_df.reset_index(drop = True)


#テストのため、行数削減
qa_df = qa_df[:100]
qa_df = qa_df.reset_index(drop = True)

print('len(qa_df)' , len(qa_df))

画像URLありの質問 除外後 206407
Answerが１wordでない 除外後 89070
len(qa_df) 100


In [4]:
#■文字
#関数：文字列→単価リスト（to_word_set～で使用）
#第二引数：「Q」＝Question用処理、「A」＝Answer用処理
def BK_sentence2wordlist(sentence , QorA):
    sentence = str(sentence)
    if QorA == 'Q':
        return list(map(lambda x : x.lower() , re.sub(r' +' , ' ' , sentence.replace(',' , ' , ').replace('.' , ' . ').replace('?' , ' ? ').replace('"' , '').replace('$' , ' $ ').replace('(' , ' ').replace(')' , ' '))))
    elif QorA == 'A':
        return sentence.lower()
      
#■単語
#関数：文字列→単価リスト（to_word_set～で使用）
#第二引数：「Q」＝Question用処理、「A」＝Answer用処理      
def sentence2wordlist(sentence , QorA):
    sentence = str(sentence)
    if QorA == 'Q':
        return list(map(lambda x : x.lower() , re.sub(r' +' , ' ' , sentence.replace(',' , ' , ').replace('.' , ' . ').replace('?' , ' ? ').replace('"' , '').replace('$' , ' $ ').replace('(' , ' ').replace(')' , ' ')).split(' ')))
    elif QorA == 'A':
        return sentence.lower()

#関数：文字列のリスト→単語の集合（重複無）※Question用
def to_word_set_Qustion(sentence_list):
    word_split_list = []
    for sentence in sentence_list:
        word_split_list += sentence2wordlist(sentence , 'Q')
    return set(word_split_list)

#関数：文字列のリスト→単語の集合（重複無）※Answer用
def to_word_set_Answer(sentence_list):
    word_split_list = []
    for sentence in sentence_list:    
        word_split_list.append(sentence2wordlist(sentence , 'A'))
    return set(word_split_list)
  
#単語辞書作成
#--単語一覧
q_word_set = to_word_set_Qustion(qa_df[' Question'])
a_word_set = to_word_set_Answer(qa_df[' Answer'])

# word_set = q_word_set | a_word_set
# del q_word_set , a_word_set

#Q用：word2id_q、id2word_q　　A用：word2id_a、id2word_a
#--Q用
word_list_q = sorted(list(q_word_set))
word2id_q = {word_list_q[i] : i for i in range(len(word_list_q))}
id2word_q = {i : word_list_q[i] for i in range(len(word_list_q))}

del q_word_set , word_list_q

#--A用
word_list_a = sorted(list(a_word_set))
word2id_a = {word_list_a[i] : i for i in range(len(word_list_a))}
id2word_a = {i : word_list_a[i] for i in range(len(word_list_a))}

del a_word_set , word_list_a

print('word2id_q' , len(word2id_q))
print('word2id_a' , len(word2id_a))

# word_list = sorted(list(word_set))
# word2id = {word_list[i] : i for i in range(len(word_list))}
# id2word = {i : word_list[i] for i in range(len(word_list))}
# del word_set , word_list

word2id_q 706
word2id_a 98


In [67]:
#モデルの型
HIDDEN_SIZE = 128
NUM_ITERATIONS = 25
SEQLEN = 10
BATCH_SIZE = 4
NUM_EPOCHS_PER_ITERATION = 1
VOCAB_SIZE = len(word2id_q)
EMBED_SIZE = 64

model = Sequential()
model.add(Embedding(VOCAB_SIZE
                   , output_dim = EMBED_SIZE
                   , embeddings_initializer = 'glorot_uniform'
                   , input_length = SEQLEN))
# model.add(SimpleRNN(HIDDEN_SIZE 
#                     , return_sequences = False
#                     , stateful = False
#                     #, batch_input_shape = (BATCH_SIZE , None , len(word2id_q)))
#                     , input_shape = (None , len(word2id_q)))
#                    )
model.add(LSTM(HIDDEN_SIZE 
                     , return_sequences = False 
                     , input_shape = (None , len(word2id_q))))
model.add(Dense(len(word2id_a)))
model.add(Activation("softmax"))

model.compile(loss = "categorical_crossentropy" , optimizer = "rmsprop")

#入力値として、各Questionに対して、SEQLEN文字ずつ区切ったものを作成し、Answerをラベルとする。
one_fit_qa_num = 1

word_split_question_list = []
word_split_answer_list = []

print(model.summary())

start = datetime.now()

#テスト
for test_no in range(32):
    for qa_no in range(len(qa_df)):

        #テスト
        print('■' * 10 , 'qa_no :' , qa_no)
        if qa_no >= 5:
            break

        word_split_question_list.append( sentence2wordlist(qa_df[' Question'][qa_no] , 'Q') )
        word_split_answer_list.append( sentence2wordlist(qa_df[' Answer'][qa_no] , 'A') )

        if ((0 if one_fit_qa_num == 1 else qa_no % (one_fit_qa_num - 1)) == 0 or qa_no == len(qa_df) - 1) and (True if one_fit_qa_num == 1 else qa_no != 0) :
            #入力用
            word_per_SEQLEN = []
            #ラベル
            y = []

            for word_split_question , word_split_answer in zip(word_split_question_list , word_split_answer_list):
                append_y_size = 0

                #入力
                if (len(word_split_question) - SEQLEN + 1) > 0:
                    for i in range(len(word_split_question) - SEQLEN + 1):
                        word_per_SEQLEN.append(word_split_question[i : i + SEQLEN])
                        append_y_size += 1
                else :
                    word_per_SEQLEN.append(word_split_question + ['' for i in range(SEQLEN - len(word_split_question))])
                    append_y_size += 1

                #ラベル
                y = y + [[1 if j == word2id_a[word_split_answer] else 0 for j in range(len(word2id_a))] for i in range(append_y_size)]

            #テスト
#             print('■word_split_question_list')
#             for i in range(len(word_split_question_list)):
#                 print(word_split_question_list[i])
#             else :
#                 print()
#             print('■word_per_SEQLEN')
#             for i in range(len(word_per_SEQLEN)):
#                 print(word_per_SEQLEN[i])
#             else:
#                 print()
#             print('■y')
#             for i in range(len(y)):
#                 print('index :' , y[i].index(1))
#             else:
#                 print('word' , id2word_a[y[i].index(1)])
#                 print()

            #※idが要素
            X = np.zeros((len(word_per_SEQLEN) , SEQLEN) , dtype = np.int)
            
            #※idが要素
            for i in range(len(word_per_SEQLEN)):
                w_list = word_per_SEQLEN[i]
                for j in range(SEQLEN):
                    wid = word2id_q[w_list[j]]
                    X[i , j] = wid
  
            #※one_hot作成
            #X = np.zeros((len(word_per_SEQLEN) , SEQLEN , len(word2id_q)) , dtype = np.bool)             
            
            #※one_hot作成
            #for i in range(len(word_per_SEQLEN)):
                #w_list = word_per_SEQLEN[i]
                #for j in range(SEQLEN):
                    #wid = word2id_q[w_list[j]]
                    #X[i , j , wid] = 1

            #×　ラベルの値
            #×　y = [[1 if j == word2id[word_split_answer] else 0 for j in range(len(word2id))] for i in range(len(word_per_SEQLEN))]  

            #学習
            print('BATCH_SIZE:' , BATCH_SIZE)
            model.fit(np.array(X) , np.array(y) , batch_size = BATCH_SIZE , epochs = NUM_EPOCHS_PER_ITERATION , shuffle = True)

            model.reset_states()            

            #1学習の時間
            end = datetime.now()
            print('learn time:' , end - start)

            #初期化
            word_split_question_list = []
            word_split_answer_list = []

            start = datetime.now()

            #if qa_no == (one_fit_qa_num - 1):
                #print('X' , np.array(X).shape)
                #print('y' , len(y))
                #break

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 10, 64)            45184     
_________________________________________________________________
lstm_13 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dense_13 (Dense)             (None, 98)                12642     
_________________________________________________________________
activation_13 (Activation)   (None, 98)                0         
Total params: 156,642
Trainable params: 156,642
Non-trainable params: 0
_________________________________________________________________
None
■■■■■■■■■■ qa_no : 0
BATCH_SIZE: 4
Epoch 1/1
10/10 [==============================] - 2s 191ms/step - loss: 4.5213
learn time: 0:00:02.995339
■■■■■■■■■■ qa_no : 1
BATCH_SIZE: 4
Epoch 1/1
11/11 [==============================] - 0s 9ms/step - loss: 4.5347
learn ti

In [68]:
#■関数

#※one_hot作成
#wordのリスト　→　入力用行列（1 × 入力word数 × len(word2id_q)）
#※各word が word2id_q に存在すること（＝絶対条件：小文字）
def BK_create_X(word_list , word2id_q = word2id_q):
    wid_list = [word2id_q[word] for word in word_list]
    return_X = np.zeros((1 , len(word_list) , len(word2id_q)) , dtype = np.bool)
    for j in range(len(wid_list)):
        return_X[0 , j , wid_list[j]] = 1
    return return_X

#※単語idが要素
#wordのリスト　→　入力用行列（1 × 入力word数 × len(word2id_q)）
#※各word が word2id_q に存在すること（＝絶対条件：小文字）
def create_X(word_list , word2id_q = word2id_q):
    wid_list = [word2id_q[word] for word in word_list]
    return_X = np.zeros((1 , len(word_list)) , dtype = int)
    for j in range(len(word_list)):
        return_X[0 , j] = wid_list[j]
    return return_X

#test_word_list = list('For the last 8 years of his life, Galileo was'.lower())
#test_word_list = ['f', 'o', 'r', ' ', 't', 'h', 'e', ' ', 'l', 'a', 's', 't', ' ', '8', ' ', 'y', 'e', 'a', 'r', 's', ' ', 'o', 'f', ' ', 'h', 'i', 's', ' ', 'l', 'i', 'f', 'e', ' ', ',', ' ', 'g', 'a', 'l', 'i', 'l', 'e', 'o', ' ', 'w', 'a', 's', ' ', 'u', 'n', 'd', 'e', 'r', ' ', 'h', 'o', 'u', 's', 'e', ' ', 'a', 'r', 'r', 'e', 's', 't', ' ', 'f', 'o', 'r', ' ', 'e', 's', 'p', 'o', 'u', 's', 'i', 'n', 'g', ' ', 't', 'h', 'i', 's', ' ', 'm', 'a', 'n', "'", 's', ' ', 't', 'h', 'e', 'o', 'r', 'y'] #copernicus
#test_word_list = ['in', '1963', ',', 'live', 'on', 'the', 'art', 'linkletter', 'show', ',', 'this', 'company', 'served', 'its', 'billionth', 'burger'] #mcdonald's
test_word_list = ['for', 'the', 'last', '8', 'years', 'of', 'his', 'life', ',', 'galileo', 'was', 'under', 'house', 'arrest', 'for', 'espousing', 'this', "man's", 'theory'] #copernicus
#test_word_list = ['the', 'city', 'of', 'yuma', 'in', 'this', 'state', 'has', 'a', 'record', 'average', 'of', '4', ',', '055', 'hours', 'of', 'sunshine', 'each', 'year'] #arizona
#test_word_list = ['a' , 'small' , 'demon' , ',' , 'or' , 'a' , 'mischievous' , 'child' , 'who' , 'might' , 'be' , 'a' , 'little' , 'demon'] #imp
#test_word_list = ['in' , 'the' , 'mid-18th' , 'c' , '.' , 'mikhail' , 'lomonosov' , 'became' , 'the' , 'first' , 'scientist' , 'to' , 'record' , 'the' , 'freezing' , 'of' , 'this' , 'liquid' , 'metal'] #mercury

print(test_word_list)

n = 3

test_X = create_X(test_word_list)
print(test_X)
test_X = np.array([test_X[0][n : n+SEQLEN]])

print(test_X)

# start_no = 0
# test_X = create_X(test_word_list[start_no : start_no + SEQLEN])

pred = model.predict(test_X , verbose = 0)[0]

['for', 'the', 'last', '8', 'years', 'of', 'his', 'life', ',', 'galileo', 'was', 'under', 'house', 'arrest', 'for', 'espousing', 'this', "man's", 'theory']
[[270 623 363  52 696 448 312 375   5 277 665 652 321  96 270 237 629 401
  626]]
[[ 52 696 448 312 375   5 277 665 652 321]]


In [69]:
out_top_n = 10

enum_pred = list(enumerate(pred))
enum_pred.sort(key = lambda x : -x[1])

top_enum_pred = enum_pred[:out_top_n]
for i in range(len(top_enum_pred)):
    print((i+1) , '位' , id2word_a[top_enum_pred[i][0]] , top_enum_pred[i][1] )

print(id2word_a[np.argmax(pred)])

1 位 copernicus 0.9999988
2 位 arizona 7.6019677e-07
3 位 washington 3.2263526e-07
4 位 mcdonald's 3.200813e-08
5 位 enamel 4.838371e-09
6 位 blossoms 3.550857e-09
7 位 helen 3.325925e-09
8 位 india 2.92704e-09
9 位 geronimo 2.91934e-09
10 位 speed 2.8340423e-09
copernicus


In [0]:
#重み取得
merge_layer = model.layers[0]
#embed_model = merge_layer.layers[0]
#word_embed_layer = word_model.layers[0]
weights = merge_layer.get_weights()[0]

pd.DataFrame(weights).to_csv('test.csv')

In [0]:
files.download('test.csv')

In [7]:
qa_df.head(5)

,Question,Answer
0,"For the last 8 years of his life, Galileo was ...",Copernicus
1,The city of Yuma in this state has a record av...,Arizona
2,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
3,"In the winter of 1971-72, a record 1,122 inche...",Washington
4,"This company's Accutron watch, introduced in 1...",Bulova


In [50]:
id2word_a

{0: '"aida"',
 1: '"candyman"',
 2: '"carmen"',
 3: '"crazy"',
 4: '"dune"',
 5: '"electricity"',
 6: '"fallin\'"',
 7: '"goose"',
 8: '"greensleeves"',
 9: '"honey"',
 10: '"jabberwocky"',
 11: '"layla"',
 12: '"mandy"',
 13: '"moondance"',
 14: '"nevermore!"',
 15: '"pagliacci"',
 16: '"rigoletto"',
 17: '"rope-a"',
 18: '"scream"',
 19: '-1',
 20: '1',
 21: '1-900',
 22: '100%',
 23: '11',
 24: '12',
 25: '14',
 26: '1865',
 27: '1930s',
 28: '1941',
 29: '1944',
 30: '1960',
 31: '1970s',
 32: '1972',
 33: '1984',
 34: '19th',
 35: '2,070',
 36: '20',
 37: '21',
 38: '24',
 39: '26',
 40: '3',
 41: '35mm',
 42: '360',
 43: '4',
 44: '48',
 45: '5',
 46: '6',
 47: '750',
 48: '8',
 49: '84',
 50: '911',
 51: '98',
 52: 'aaron',
 53: 'abscam',
 54: 'academia',
 55: 'acadia',
 56: 'acapulco',
 57: 'accommodations',
 58: 'accordion',
 59: 'achievement',
 60: 'acquired',
 61: 'acura',
 62: 'adenoids',
 63: 'adieu',
 64: 'adonis',
 65: 'aerosmith',
 66: 'aeschylus',
 67: 'aesop',
 68: 'a

In [30]:
#Answer が 1word　でない　を除外する前
print(len(word2id))

9312


In [37]:
#Answer が 1word　でない　を除外した後
print(len(word2id))

8822


In [40]:
print(word2id['last'])
print(word2id['8'])
print(word2id['years'])
print(word2id['of'])
print(word2id['his'])


print(word2id['galileo'])
print(word2id['was'])


4933
604
9254
5997
4220
3724
8958


In [24]:
n = len(qa_df) - 1

print(qa_df[' Question'][n])
print(qa_df[' Answer'][n])

In the mid-18th c. Mikhail Lomonosov became the first scientist to record the freezing of this liquid metal
mercury


In [8]:
a_word_set
#word2id

{'(omar) bradley',
 'joseph smith',
 'brussels',
 'black death',
 'jakarta',
 'the new world',
 'medellin',
 'pods',
 'jim carrey',
 'ordinary people',
 'california',
 'drooling',
 'soft',
 'walt disney',
 'milkweed',
 'the nobel prizes',
 'why does swiss cheese have holes?',
 'the emerald city',
 '750',
 'tnt',
 'alexander graham bell',
 'arms',
 'bunker',
 'old mother hubbard',
 'rhizomes',
 '\\"death on the nile\\"',
 'easter island',
 '"rope-a"',
 'the indy 500',
 'harold stratton',
 'steadicam',
 'tooth',
 'coldplay',
 'prologue',
 'oh! calcutta!',
 'carl sandburg',
 'tiger woods',
 'gloves',
 'tee time',
 'urban outfitters',
 'psycho',
 "the (runnin') rebels",
 'maids',
 'kalahari',
 'abraham lincoln',
 'the canterbury tales',
 'zoroastrianism',
 'laugh track',
 'lake champlain',
 'hart to hart',
 'chris rock',
 'why does the snake crawl on the ground?',
 'prince william',
 'john cheever',
 '(frank) gehry',
 'august wilson',
 '"the village blacksmith"',
 'helen',
 '(art) garfunke